## I. 패키지 밎 자료 불러오기

In [ ]:
!pip install pymysql
!pip install sqlalchemy
!pip install mysqlclient

     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 87 kB 3.6 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99962 sha256=53b3d2a1a9e4c2455a310a4aa4bd82071f32dbfc2f8a8307b2730b1ffddb2e50
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
Successfully built mysqlclient


In [ ]:
!pip install pyecharts
!pip install chart_studio

     |████████████████████████████████| 135 kB 5.1 MB/s 
     |████████████████████████████████| 130 kB 53.3 MB/s 
     |████████████████████████████████| 64 kB 2.0 MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import time
import datetime
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc

import itertools
import operator

import base64
from io import BytesIO

from datetime import datetime, timedelta

from pyecharts.charts import Bar
from pyecharts.charts import Line
import plotly.offline as plyo
import cufflinks
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
db = pymysql.connect(host=, charset='utf8')
cursor = db.cursor(pymysql.cursors.DictCursor)
sql = 'SELECT * FROM cj'
cursor.execute(sql)
result = cursor.fetchall()
DF = pd.DataFrame(result)


In [ ]:
DF.tail()

,index,date,coin_name,s,s.1,lang
2611,20,2021-11-27,snt,2,143,eng
2612,21,2021-11-27,snt,0,57,eng
2613,22,2021-11-27,xrp,2,6397,eng
2614,23,2021-11-27,xrp,0,3142,eng
2615,24,2021-11-27,xrp,1,14,eng


In [ ]:
# 저장 포인트
DF.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/sentiment_result_db_2.csv')

## II. 일단 긍정/중립/부정 3개의 데이터 프래임으로 모으기
- 없을시 0으로 해야 함으로 데이터 프레임을 먼저 만들고 거기에 합치기
- s.1은 갯수

In [ ]:
# date 자동화 for문 작업은 나중에 -> DF['date']의 원소를 리스트 형식으로 받아 date으로 정의
date = ['2021-11-17','2021-11-18','2021-11-19','2021-11-20','2021-11-21','2021-11-22',
        '2021-11-23','2021-11-24','2021-11-25','2021-11-26','2021-11-27']
        # ,'2021-11-28','2021-11-29','2021-11-30']
# lang = ['kor','eng']
crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']
s = [0,1,2]

### 1) 새로운 데이터 프레임을 만들어야한다.

In [ ]:
# 새로운 데이터 프레임 만들기
df = pd.DataFrame(columns = ['date', 'crypto_name', 'positive_E','neutrual_E','negative_E','positive_K','neutrual_K','negative_K'])
for i in date:
  for j in crpyto_name:
    df = df.append(pd.DataFrame([[i, j]], columns = ['date','crypto_name']), ignore_index=True)
df

,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
0,2021-11-17,ada,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-11-17,bch,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-11-17,btc,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-11-17,doge,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-11-17,eos,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
94,2021-11-27,eos,NaN,NaN,NaN,NaN,NaN,NaN
95,2021-11-27,eth,NaN,NaN,NaN,NaN,NaN,NaN
96,2021-11-27,omg,NaN,NaN,NaN,NaN,NaN,NaN
97,2021-11-27,snt,NaN,NaN,NaN,NaN,NaN,NaN


### 2) 새로운 데이터 프레임에 값들 집어 넣기 - 모든 코인 모든 날짜

In [ ]:
for k in crpyto_name:

  for j in date:

    for i in range(len(DF)):


      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 0):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['positive_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 0):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['positive_E'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 1):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['neutrual_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 1):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['neutrual_E'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 2):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['negative_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 2):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['negative_E'][a] = DF['s.1'][i]    

------ 해킹당한 날 빽업 파일 붙여넣기

In [ ]:
db = pymysql.connect(host=, charset='utf8')
cursor = db.cursor(pymysql.cursors.DictCursor)
sql = 'SELECT * FROM graphing_backup_Nov23'
cursor.execute(sql)
result = cursor.fetchall()
DF_backup = pd.DataFrame(result)


In [ ]:
for k in crpyto_name:

  for j in date:

    for i in range(len(DF_backup)):


      if (DF_backup['date'][i] == j) and (DF_backup['crypto_name'][i] == k):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)].tolist()

          df['positive_E'][a] = DF_backup['positive_E'][i]
          df['neutrual_E'][a] = DF_backup['neutrual_E'][i]
          df['negative_E'][a] = DF_backup['negative_E'][i]
          df['positive_K'][a] = DF_backup['positive_K'][i]
          df['neutrual_K'][a] = DF_backup['neutrual_K'][i]
          df['negative_K'][a] = DF_backup['negative_K'][i]


----- 여기까지

In [ ]:
df = df.fillna(0)

In [ ]:
df.tail(20)

,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
79,2021-11-25,snt,2,0,6,2,0,0
80,2021-11-25,xrp,337,0,726,32,11,16
81,2021-11-26,ada,47447,170,108637,0,1,1
82,2021-11-26,bch,135,0,238,1,0,0
83,2021-11-26,btc,6324,16,10767,0,0,0
84,2021-11-26,doge,4061,19,7360,0,0,1
85,2021-11-26,eos,350,1,627,0,0,0
86,2021-11-26,eth,24394,82,39804,0,0,0
87,2021-11-26,omg,1,0,9,0,0,0
88,2021-11-26,snt,53,0,101,0,0,0


## --------- 전처리 작업 완료

In [ ]:
# 저장포인트
df.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/sentiment_result_graphing.csv')

In [ ]:
# 저장포인트
daily_engine = create_engine("mysql+mysqldb:", encoding='utf-8',pool_size = 100000,max_overflow = 0)
daily_conn = daily_engine.connect()
df.to_sql(name='Over_all',con=daily_conn,if_exists='replace')
print('업데이트 완료 : Over_all', '\n' )

업데이트 완료 : Over_all 



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/sentiment_result_graphing.csv')

## --------- DB와 코렙에 저장 포인트

## III. 지수화
- 일단 긍정의 전체비율만을 가지고 한다. (대부분의 경우 상승에 관심 있고 롱 포지션을 가지기때문에)
- 영어가 대부분이고 영어는 거의 중립이 없어서 가능할지도
- 나중에 부정의 비율도 고려한다.


In [ ]:
date = ['2021-11-17','2021-11-18','2021-11-19','2021-11-20','2021-11-21','2021-11-22',
        '2021-11-23','2021-11-24','2021-11-25','2021-11-26','2021-11-27']
        # ,'2021-11-28','2021-11-29','2021-11-30']
# lang = ['kor','eng']
crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']
s = [0,1,2]

### 1) 긍정비율

In [ ]:
df['positive_ratio_E'] = df['positive_E']/(df['positive_E']+df['neutrual_E']+df['negative_E'])
df['positive_ratio_K'] = df['positive_K']/(df['positive_K']+df['neutrual_K']+df['negative_K'])

In [ ]:
df.head(30)

,Unnamed: 0,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
0,0,2021-11-17,ada,37535,124,97792,6,7,3,0.277111,0.375000
1,1,2021-11-17,bch,67,0,152,20,5,10,0.305936,0.571429
2,2,2021-11-17,btc,5040,17,12443,3,2,1,0.288000,0.500000
3,3,2021-11-17,doge,2125,11,5339,10,4,4,0.284281,0.555556
4,4,2021-11-17,eos,304,2,662,13,8,17,0.314050,0.342105
5,5,2021-11-17,eth,14031,45,36835,0,0,0,0.275599,NaN
6,6,2021-11-17,omg,13,0,26,0,0,1,0.333333,0.000000
7,7,2021-11-17,snt,34,0,58,0,1,1,0.369565,0.000000
8,8,2021-11-17,xrp,1593,3,4019,13,2,10,0.283704,0.520000
9,9,2021-11-18,ada,0,0,0,1,1,2,NaN,0.250000


In [ ]:
df.tail(30)

,Unnamed: 0,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
69,69,2021-11-24,omg,3,0,6,0,1,0,0.333333,0.000000
70,70,2021-11-24,snt,27,0,101,0,0,1,0.210938,0.000000
71,71,2021-11-24,xrp,2986,11,7441,35,8,21,0.286070,0.546875
72,72,2021-11-25,ada,7542,18,16138,7,3,9,0.318255,0.368421
73,73,2021-11-25,bch,21,0,44,10,4,3,0.323077,0.588235
74,74,2021-11-25,btc,727,1,1461,1,0,0,0.332115,1.000000
75,75,2021-11-25,doge,787,0,1429,7,7,7,0.355144,0.333333
76,76,2021-11-25,eos,43,0,66,26,16,21,0.394495,0.412698
77,77,2021-11-25,eth,2720,9,5001,2,0,0,0.351876,1.000000
78,78,2021-11-25,omg,0,0,2,0,0,0,0.000000,NaN


## IV. 암호화폐별 추출 밎 가격테이블 연결

### 1) 가격데이터, 감정데이터 불러오기; 각코인별 데이터 프레임 만들기

In [ ]:
crypto_name_p = ['ADA','BTC','BCH','DOGE','EOS','ETH','OMG','SNT','XRP']

for i in crypto_name_p:
    db = pymysql.connect(host=, charset='utf8')
    cursor = db.cursor(pymysql.cursors.DictCursor)
    sql = 'SELECT * FROM {0}'.format(i)
    cursor.execute(sql)
    result = cursor.fetchall()
    globals()[f'{i}'] = pd.DataFrame(result)



crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]
crypto_name_p = ['ADA','BTC','BCH','DOGE','EOS','ETH','OMG','SNT','XRP']

for j1,j2,z1 in zip(crypto_p,crypto_p,crypto_name_p):

  X = []

  for i in range(len(j2)):
    date_time_obj = datetime.strftime(j2['index'][i], '%Y-%m-%d')
    a = date_time_obj
    b = j2['close'][i]
    # c = j2['value'][i]
    d = j2['volume'][i]
    e = 0
    f = 0
    g = 0
    h = 0
    i = 0
    j = 0
    k = 0
    l = 0

    x = [a,b,d,e,f,g,h,i,j,k,l]
    X.append(x)
  globals()['{0}'.format(z1)] = pd.DataFrame(X)
  globals()['{0}'.format(z1)].columns = ['date','close_KRW','volume','positive_E','neutrual_E','negative_E',	'positive_K',	'neutrual_K',	'negative_K','positive_ratio_E','positive_ratio_K']

In [ ]:
XRP.tail(20)

,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
1504,2021-11-10,1460.0,7.085848e+08,0,0,0,0,0,0,0,0
1505,2021-11-11,1480.0,3.418756e+08,0,0,0,0,0,0,0,0
1506,2021-11-12,1450.0,2.971702e+08,0,0,0,0,0,0,0,0
1507,2021-11-13,1440.0,1.428960e+08,0,0,0,0,0,0,0,0
1508,2021-11-14,1440.0,1.156236e+08,0,0,0,0,0,0,0,0
1509,2021-11-15,1425.0,2.039552e+08,0,0,0,0,0,0,0,0
1510,2021-11-16,1340.0,4.614623e+08,0,0,0,0,0,0,0,0
1511,2021-11-17,1340.0,3.449736e+08,0,0,0,0,0,0,0,0
1512,2021-11-18,1285.0,6.187245e+08,0,0,0,0,0,0,0,0
1513,2021-11-19,1345.0,2.626295e+08,0,0,0,0,0,0,0,0


### 2) 가격데이터에 감정데이터 붙여 넣기
- 가격데이터가 더많이 있어서

In [ ]:
crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]
crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']

for k,l in zip(crypto_p,crpyto_name):

  for j in date:

      a = df.index[(df['date'] == j)&(df['crypto_name'] == l)].tolist()
      b = k.index[(k['date'] == j)].tolist()
      k['positive_E'][b] = df['positive_E'][a]
      k['neutrual_E'][b] = df['neutrual_E'][a]
      k['negative_E'][b] = df['negative_E'][a]
      k['positive_K'][b] = df['positive_K'][a]
      k['neutrual_K'][b] = df['neutrual_K'][a]
      k['negative_K'][b] = df['negative_K'][a]
      k['positive_ratio_E'][b] = df['positive_ratio_E'][a]
      k['positive_ratio_K'][b] = df['positive_ratio_K'][a]

In [ ]:
ADA.tail(20)

,Unnamed: 0,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
1493,1493,2021-11-10,2580.0,8.247482e+07,0,0,0,0,0,0,0.000000,0.000000
1494,1494,2021-11-11,2535.0,3.012966e+07,0,0,0,0,0,0,0.000000,0.000000
1495,1495,2021-11-12,2495.0,3.415626e+07,0,0,0,0,0,0,0.000000,0.000000
1496,1496,2021-11-13,2480.0,1.670549e+07,0,0,0,0,0,0,0.000000,0.000000
1497,1497,2021-11-14,2470.0,1.192238e+07,0,0,0,0,0,0,0.000000,0.000000
1498,1498,2021-11-15,2465.0,2.125746e+07,0,0,0,0,0,0,0.000000,0.000000
1499,1499,2021-11-16,2310.0,4.825794e+07,0,0,0,0,0,0,0.000000,0.000000
1500,1500,2021-11-17,2300.0,3.939187e+07,37535,124,97792,6,7,3,0.277111,0.375000
1501,1501,2021-11-18,2215.0,4.047463e+07,0,0,0,1,1,2,NaN,0.250000
1502,1502,2021-11-19,2300.0,2.526605e+07,38911,95,108164,3,0,1,0.264395,0.750000


## V. DB에 적제

In [ ]:
# 중간저장
ADA.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/ADA.csv')
BCH.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/BCH.csv')
BTC.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/BTC.csv')
DOGE.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/DOGE.csv')
EOS.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/EOS.csv')
ETH.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/ETH.csv')
OMG.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/OMG.csv')
SNT.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/SNT.csv')
XRP.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/XRP.csv')

In [ ]:
# 중간저장 backup
crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]

for i in crypto_name_p:
  daily_engine = create_engine("mysql+mysqldb:", encoding='utf-8',pool_size = 100000,max_overflow = 0)
  daily_conn = daily_engine.connect()
  globals()[f'{i}'].to_sql(name=f'{i}_backup',con=daily_conn,if_exists='replace')
  print('업데이트 완료 :', f'{i}_backup', '\n' )

업데이트 완료 : ADA_backup 

업데이트 완료 : BTC_backup 

업데이트 완료 : BCH_backup 

업데이트 완료 : DOGE_backup 

업데이트 완료 : EOS_backup 

업데이트 완료 : ETH_backup 

업데이트 완료 : OMG_backup 

업데이트 완료 : SNT_backup 

업데이트 완료 : XRP_backup 



In [ ]:
# 중간저장
crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]

for i in crypto_name_p:
  daily_engine = create_engine("mysql+mysqldb:", encoding='utf-8',pool_size = 100000,max_overflow = 0)
  daily_conn = daily_engine.connect()
  globals()[f'{i}'].to_sql(name=f'{i}',con=daily_conn,if_exists='replace')
  print('업데이트 완료 :', f'{i}', '\n' )

업데이트 완료 : ADA 

업데이트 완료 : BTC 

업데이트 완료 : BCH 

업데이트 완료 : DOGE 

업데이트 완료 : EOS 

업데이트 완료 : ETH 

업데이트 완료 : OMG 

업데이트 완료 : SNT 



In [ ]:
# 불러오기
ADA = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/ADA.csv')
BCH = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/BCH.csv')
BTC = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/BTC.csv')
DOGE = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/DOGE.csv')
EOS = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/EOS.csv')
ETH = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/ETH.csv')
OMG = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/OMG.csv')
SNT = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/SNT.csv')
XRP = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/XRP.csv')

In [ ]:
ADA

,Unnamed: 0,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
0,0,2017-10-06,21.3,3.330000e-02,0,0,0,0,0,0,0.000000,0.000000
1,1,2017-10-07,23.7,5.200000e-02,0,0,0,0,0,0,0.000000,0.000000
2,2,2017-10-08,23.2,5.200000e-02,0,0,0,0,0,0,0.000000,0.000000
3,3,2017-10-09,25.3,5.190000e-02,0,0,0,0,0,0,0.000000,0.000000
4,4,2017-10-10,24.7,5.140000e-02,0,0,0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1508,1508,2021-11-25,2070.0,2.482044e+07,7542,18,16138,7,3,9,0.318255,0.368421
1509,1509,2021-11-26,2065.0,2.041012e+06,47447,170,108637,0,1,1,0.303653,0.000000
1510,1510,2021-11-27,1975.0,2.015293e+07,72115,244,153426,0,0,0,0.319397,NaN
1511,1511,2021-11-28,2010.0,3.569564e+07,0,0,0,0,0,0,0.000000,0.000000
